# Client system operations Multiagent system

Este proyecto realiza operaciones sobre los clientes de una empresa.

In [1]:
%%capture --no-stderr
%pip install -U --upgrade pip langgraph langchain_community langchain_anthropic langchain-tavily langchain_experimental langchain_ollama mcp langchain-mcp-adapters

In [2]:
from langfuse.callback import CallbackHandler
# load environment variables from .env file
from dotenv import load_dotenv
load_dotenv()
# Initialize Langfuse CallbackHandler for LangGraph/Langchain (tracing)
langfuse_handler = CallbackHandler() 
 

In [15]:
from langchain_ollama import ChatOllama
from langgraph.prebuilt import create_react_agent

simple_agent = create_react_agent(name="simple_agent", model=ChatOllama(model="qwen3"), 
    tools=[], response_format='json', prompt="You are a helpful assistant. Your task is to respond to a user's question.")

input_message = {"role": "user", "content": """
Generate the python code for a function that writes a string content in a file name path given. 
the function is a langchain tool
three parameters: content , full_file_path and append (True or False)
should be able to handle exceptions and return an error message if the file cannot be written.
Function must have a docstring
use encoding utf8
give just the code, no introduction /no_think"""
}

# Use the agent
config = {"configurable": {"thread_id": "abc123"}, "recursion_limit": 10, "callbacks": [langfuse_handler]}
for step in simple_agent.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================


Generate the python code for a function that writes a string content in a file name path given. 
the function is a langchain tool
three parameters: content , full_file_path and append (True or False)
should be able to handle exceptions and return an error message if the file cannot be written.
Function must have a docstring
use encoding utf8
give just the code, no introduction /no_think
================================== Ai Message ==================================
Name: simple_agent

<think>

</think>

```python
from langchain.tools import tool

@tool
def write_to_file(content: str, full_file_path: str, append: bool = False) -> str:
    """
    Writes the given content to a file at the specified full_file_path.
    If append is True, the content is added to the end of the file.
    If False, the file is overwritten.
    Returns a success message or an error message if writing fails.
    """
    try:
  

In [17]:
from langchain_core.tools import tool 

@tool
def write_to_file(content: str, full_file_path: str, append: bool = False) -> str:
    """
    Writes the given content to a file at the specified full_file_path.
    If append is True, the content is added to the end of the file.
    If False, the file is overwritten.
    Returns a success message or an error message if writing fails.
    """
    try:
        mode = 'a' if append else 'w'
        with open(full_file_path, mode, encoding='utf-8') as file:
            file.write(content)
        return f"Successfully wrote content to {full_file_path}"
    except Exception as e:
        return f"Failed to write to file {full_file_path}. Error: {str(e)}"

In [39]:
simple_agent = create_react_agent(name="simple_agent", model=ChatOllama(model="qwen3"), 
    tools=[write_to_file], response_format='json', prompt="You are a helpful assistant. Your task is to respond to a user's question.")

input_message = {"role": "user", "content": """
generate a csv list of 50 dummy clients
with columns headers 
with client_id, name, address, phone_number, email, prefered_languaje
write the list in a file in '/home/daimler/workspaces/agents-course-huggingface/langagent/data/clients.csv'
 /no_think"""
}

# Use the agent
config = {"configurable": {"thread_id": "abc123"}, "recursion_limit": 10, "callbacks": [langfuse_handler]}
for step in simple_agent.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================


generate a csv list of 50 dummy clients
with columns headers 
with client_id, name, address, phone_number, email, prefered_languaje
write the list in a file in '/home/daimler/workspaces/agents-course-huggingface/langagent/data/clients.csv'
 /no_think
================================== Ai Message ==================================
Name: simple_agent

<think>

</think>
Tool Calls:
  write_to_file (2a3847e0-c758-4f80-9edd-cfc8ea2c3e8f)
 Call ID: 2a3847e0-c758-4f80-9edd-cfc8ea2c3e8f
  Args:
    append: False
    content: client_id,name,address,phone_number,email,prefered_languaje
1,John Doe,123 Main St,555-1234,john.doe@example.com,English
2,Jane Smith,456 Oak St,555-5678,jane.smith@example.com,Spanish
3,Bob Johnson,789 Pine St,555-9012,bob.johnson@example.com,Portuguese
4,Alice Williams,321 Maple St,555-3456,alice.williams@example.com,French
5,Michael Brown,654 Cedar St,555-7890,michael.brown@example.com,Ge

In [20]:
input_message = {"role": "user", "content": """
Generate the python code for a function that returns the client details for a client id given. 
the function is a langchain tool
one parameters: client_id
use pandas to load a csv file in '/home/daimler/workspaces/agents-course-huggingface/langagent/data/clients.csv'
find the client details in a pandas dataframe return dict with all columns
should be able to handle exceptions and return an error message if the file cannot be found.
Function must have a docstring
use encoding utf8
give just the code, no introduction /no_think"""
}

# Use the agent
config = {"configurable": {"thread_id": "abc123"}, "recursion_limit": 10, "callbacks": [langfuse_handler]}
for step in simple_agent.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()


================================ Human Message =================================


Generate the python code for a function that returns the client details for a client id given. 
the function is a langchain tool
one parameters: client_id
use pandas to load a csv file in '/home/daimler/workspaces/agents-course-huggingface/langagent/data/clients.csv'
find the client details in a pandas dataframe return dict with all columns
should be able to handle exceptions and return an error message if the file cannot be found.
Function must have a docstring
use encoding utf8
give just the code, no introduction /no_think
================================== Ai Message ==================================
Name: simple_agent

<think>

</think>

def get_client_details(client_id):
    """
    Retrieve client details from a CSV file based on the provided client ID.

    Parameters:
    client_id (str): The ID of the client to retrieve details for.

    Returns:
    dict: A dictionary containing all client det

In [59]:
import pandas as pd
@tool
def get_client_details(client_id):
    """
    Retrieve client details of the provided client ID.

    Parameters:
    client_id (str): The ID of the client to retrieve details for.

    Returns:
    dict: A dictionary containing all client details if found, otherwise an error message.
    """
    try:
        file_path = '~/workspaces/agents-course-huggingface/.workspace/clients.csv'
        df = pd.read_csv(file_path, encoding='utf-8')
        client_details = df[df['client_id'] == int(client_id)].to_dict(orient='records')
        if client_details:
            return client_details[0]
        else:
            return {"error": "Client not found"}
    except FileNotFoundError:
        return {"error": "File not found"}
    except Exception as e:
        return {"error": str(e)}

get_client_details.invoke('31')

{'client_id': 31,
 'name': 'Charles Garcia',
 'address': '112 Cedar St',
 'phone_number': '555-4567',
 'email': 'charles.garcia@example.com',
 'prefered_languaje': 'Thai'}

In [58]:
import random

simple_agent = create_react_agent(name="simple_agent", model=ChatOllama(model="qwen3"), 
    tools=[get_client_details], response_format='json', prompt="You are a helpful assistant. Your task is to respond to a user's question.")

input_message = {"role": "user", "content": f"""
get the client details of a client with id '{random.randint(0, 50)}'
generate an email explaining we could not deliver the package at his address nor contact him at his phone number.
use they prefered language
especify the actual address and phone number of the client.
/no_think"""
}

# Use the agent
config = {"configurable": {"thread_id": "abc123"}, "recursion_limit": 10, "callbacks": [langfuse_handler]}
for step in simple_agent.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================


get the client details of a client with id '40'
generate an email explaining we could not deliver the package at his address nor contact him at his phone number.
use they prefered language
especify the actual address and phone number of the client.
/no_think
================================== Ai Message ==================================
Name: simple_agent

<think>

</think>
Tool Calls:
  get_client_details (8877dc99-4c3f-4a2f-8beb-c827cd068de0)
 Call ID: 8877dc99-4c3f-4a2f-8beb-c827cd068de0
  Args:
    client_id: 40
================================= Tool Message =================================
Name: get_client_details

{"client_id": 40, "name": "Charles Garcia", "address": "321 Cedar St", "phone_number": "555-8901", "email": "charles.garcia@example.com", "prefered_languaje": "Arabic"}
================================== Ai Message ==================================
Name: simple_agent

<think>

</think>